In [3]:
import math

import numpy
import pandas

import ipywidgets as widgets
from IPython.display import display, clear_output

import gensim
from gensim.corpora import Dictionary
from gensim.models import TfidfModel
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

import pythainlp

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans

from data_tokenizer import load_corpus

from model.new_sdc import NewSDC
from model.sdc import SDC

### Load Data

In [4]:
file_name = 'ผู้บริโภค - TrueCoffee'

corpus, labels = load_corpus('../data/' + file_name + '.txt')

len_corpus = len(corpus)
print('Total documents', len_corpus)

clusters = list(set(labels))
print(len(clusters), 'clusters')

f = open('../data/tokenized/tokenized_' + file_name + '.txt')
tokenized_corpus = eval(f.read())
f.close()

Total documents 350
1 clusters


### Preprocess Corpus

#### Remove Words

In [5]:
dictionary = Dictionary(tokenized_corpus)
print('origin:', len(dictionary), 'words')

dictionary.filter_extremes(no_below=2, no_above=0.7, keep_n=len(dictionary))
print('filter frequent words:', len(dictionary), 'words')

letter_words = [id for id in range(len(dictionary)) if len(dictionary[id]) <= 1] 
dictionary.filter_tokens(bad_ids=letter_words)
print('filter letter words:', len(dictionary), 'words')

stopwords = pythainlp.corpus.stopwords.words('thai')
stopwords.extend(['นี้'])
dictionary.add_documents([stopwords])
stopwords = [dictionary.token2id[word] for word in stopwords]
dictionary.filter_tokens(bad_ids=stopwords)
print('filter stop words:', len(dictionary), 'words')

origin: 948 words
filter frequent words: 370 words
filter letter words: 368 words
filter stop words: 219 words


In [6]:
idx_corpus = [dictionary.doc2idx(doc) for doc in tokenized_corpus]

temp_corpus = []
for doc in idx_corpus:
    temp_corpus.append([dictionary[id] for id in doc if id >= 0])
idx_corpus = temp_corpus

#### Dimension Reduction

In [7]:
average_doc_size = 0
for doc in idx_corpus:
    average_doc_size += len(doc)
average_doc_size /= len(idx_corpus)
average_doc_size = math.ceil(average_doc_size)

df = dictionary.dfs
filtered_corpus = []
for doc in idx_corpus:
    new_doc = [(word, df[dictionary.token2id[word]]) for word in doc]
    new_doc.sort(reverse=True, key=lambda x: x[1])
    new_doc = new_doc[:average_doc_size]
    filtered_corpus.append([word for word, df in new_doc])

#### Doc2Vec

In [8]:
tagged_corpus = [TaggedDocument(doc, [i]) for i, doc in enumerate(idx_corpus)]
model = Doc2Vec(tagged_corpus, vector_size=average_doc_size, window=4, min_count=2, epochs=100)
model.delete_temporary_training_data(keep_doctags_vectors=True, keep_inference=True)

paragraph_vectors = [model.infer_vector(doc) for doc in idx_corpus]
paragraph_vectors = pandas.DataFrame(paragraph_vectors, dtype=float)

### Clustering

In [9]:
def get_onehot(corpus, weight):
    dictionary = Dictionary(corpus)
#     dictionary.filter_extremes(no_below=2, no_above=1, keep_n=len(dictionary))

    bow_corpus = [dictionary.doc2bow(doc) for doc in corpus]
    if weight == 'normal':
        weight_corpus = bow_corpus
    elif weight == 'tfidf':
        tfidf = TfidfModel(bow_corpus, smartirs='ltc')
        weight_corpus = [tfidf[doc] for doc in bow_corpus]

    unique_words = [dictionary[id] for id in range(len(dictionary))]
    array = numpy.zeros((len(corpus), len(unique_words)), dtype=float)
    for i, doc in enumerate(weight_corpus):
        for id, score in doc:
            array[i, id] = score

        if weight == 'normal' and len(doc) != 0:
#             array[i] = numpy.divide(array[i], len(idx_corpus[i]))
            array[i] = numpy.divide(array[i], len(doc))
    
    return pandas.DataFrame(array, columns=unique_words, dtype=float)

In [10]:
def generate_result(predicted_labels, marks):
    result = pandas.DataFrame()
    result['comment'] = corpus
    result['tokenized_comment'] = idx_corpus
    result['label'] = labels
    result['predicted_label'] = predicted_labels
    if marks:
        result['marks'] = marks
    else:
        result['marks'] = -1
    return result

In [11]:
def eval_cluster(onehot_corpus, result):
    label_count = numpy.unique(result['predicted_label'])
    num_cluster = label_count[-1] + 1

    clusters = [[] for i in range(num_cluster)]
    corpus_centroid = []
    for i, label in result['predicted_label'].iteritems():
        clusters[label].append(numpy.array(onehot_corpus.iloc[i]))
        corpus_centroid.append(numpy.array(onehot_corpus.iloc[i]))
    corpus_centroid = numpy.mean(corpus_centroid, axis=0).reshape(1, -1)   

#     print('\tIntra cluster sim\tInter cluster sim\tIntra / Inter')
    compactness = 0
    centroids = []
    for i in range(num_cluster):
        size = len(clusters[i])
        if size != 0:
            centroid = numpy.mean(clusters[i], axis=0)
            centroids.append(centroid)
            centroid = centroid.reshape(1, -1)
            similarities = cosine_similarity(centroid, clusters[i])
            compactness += numpy.sum(similarities)

#             intra = numpy.sum(similarities) / size
#             inter = cosine_similarity(centroid, corpus_centroid)[0][0]
#             print(i, end='\t')
#             print(intra, end='\t')
#             print(inter, end='\t')
#             print(intra / inter)
    return compactness, centroids

In [12]:
min_samples = 7
eps = 0.32
expand_rate = 0.05

onehot_corpus = get_onehot(idx_corpus, 'normal')
# onehot_corpus = get_onehot(filtered_corpus, 'tfidf')

In [21]:
max_compactness = 0
epoch = 15
for i in range(epoch):
#     model = NewSDC()
#     _tpredicted_labels, marks = model.predict(onehot_corpus, min_samples, eps, expand_rate)

#     model = SDC()
#     _tpredicted_labels, marks = model.predict(onehot_corpus, min_samples, eps)
    
    marks = None
    
#     model = DBSCAN(metric='cosine', eps=eps, min_samples=min_samples).fit(onehot_corpus)
#     _tpredicted_labels = model.labels_ + 1

    model = KMeans(n_clusters=14).fit(onehot_corpus)
    _tpredicted_labels = model.labels_
    
    _tresult = generate_result(_tpredicted_labels, marks)
    compactness, _tcentroids = eval_cluster(onehot_corpus, _tresult)
    
    if compactness > max_compactness:
        max_compactness = compactness
        predicted_labels = _tpredicted_labels
        result = _tresult
        centroids = _tcentroids
        
print(max_compactness)
label_count = numpy.unique(result['predicted_label'], return_counts=True)[1]
num_cluster = len(label_count)
print(label_count)

163.1837864203253
[ 30  10  30   9  12  12  25  22  10  10  25 149   4   2]


In [ ]:
centroids = None 
prev_label_count = None
while True:
    model = NewSDC()
    predicted_labels, marks = model.predict(onehot_corpus, min_samples, eps, expand_rate, seeds=centroids)
    
    result = generate_result(predicted_labels, marks)
    compactness, centroids = eval_cluster(onehot_corpus, result)
    
    label_count = numpy.unique(result['predicted_label'], return_counts=True)[1]
    if numpy.array_equal(label_count, prev_label_count):
        break
    prev_label_count = label_count
    centroids = centroids[1:]
    
    print(compactness)
    print(label_count)
num_cluster = len(label_count)

In [22]:
sims = cosine_similarity(centroids)
new_labels = [i for i in range(num_cluster)]
print(new_labels)
for i, row in reversed(list(enumerate(sims))):
    for j, value in reversed(list(enumerate(row[:i + 1]))):
        if i != j and value >= eps - eps / 20:
            print(i, j, value)
            base = min(new_labels[i], new_labels[j])
            new_labels[j] = base
            new_labels = [base if label == new_labels[i] else label for label in new_labels]
print(new_labels)

grouped_labels = numpy.zeros(len_corpus)
for i, label in enumerate(predicted_labels):
    grouped_labels[i] = new_labels[label]
new_result = generate_result(grouped_labels, None)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
11 1 0.30535198048644013
10 9 0.8783020838381587
10 3 0.34298458536886484
3 2 0.33847700234811184
[0, 1, 2, 2, 4, 5, 6, 7, 8, 2, 2, 1, 12, 13]


### Result

In [23]:
class Widget:
    def __init__(self, result, column_name):
        self.result = result
        self.column_name = column_name
        
        label_count = numpy.unique(result['predicted_label'])
        self.widget = widgets.ToggleButtons(
            options=[int(num) for num in label_count],
            disabled=False,
            button_style='',
        )
        
        self.widget.observe(self.on_click, names='index')
        self.on_click({'new' : 0})
        
    def on_click(self, change):
        clear_output()
        display(self.widget)
        new = self.widget.options[change['new']]
        for index, value in self.result[self.result['predicted_label'] == new].iterrows():
            if value['marks'] == 0:
                print("@", end="")
            elif value['marks'] == 1:
                print("*", end="")
            print(index, value[self.column_name])

In [24]:
result.to_csv('../data/results/k-mean/' + file_name + '.csv')

# result = pandas.read_csv('../data/results/k-mean/' + file_name + '.csv')

count = 0
for index, value in result.iterrows():
    if value['marks'] == -1:
        count += 1
print(count)

350


In [25]:
w1 = Widget(new_result, 'comment')

ToggleButtons(options=(0, 1, 2, 4, 5, 6, 7, 8, 12, 13), value=0)

6 ไม่ ชอบ การ แบ่ง ชนชั้น ในร้านกาแฟ ผู้ถือบัตรอีกแบบ นั่งได้แค่ตรงนี้ ผู้ถือบัตรอีกแบบนั่งตรงนี้ จ่ายราคากาแฟก็เท่ากันไหม
57 เป็นร้านที่แดกกาแฟร้อนทีไร แม่งต้องล้วกลิ้นไหม้เป็นแผลตลอด น้ำร้อนเอ็งจะร้อนไปไหนวะ สงสัยแม่งเอามาจากบ่อในนรกชัวร์ๆ ถ้าเอ็งร้อนแบบนี้
58 ทานที่มหาวิทยาลัยเป็นมุมกาแฟรสชาติอร่อยมาก กาแฟดีมาก ราคาดีรับได้ แต่ถ้าเป็นร้านในห้างราคา over เกินไป
90 กาแฟพอใช้ได้นะ แต่ราคาก็ยังสูงกว่าร้านอื่นๆอยุ่ ลงราคามาหน่อย หรือจัดโทรโมชั่นบ้าง
99 ลองเปลี่ยนไปขายอย่างอื่นที่ไม่ใช่กาแฟดีกว่า
101 กาแฟแม่งแรงสัสมึงจะดีดไปไหน
114 กาแฟจืดสัดๆ เปรี้ยวด้วย ใช้โรบัสต้า ใช่มั้ยมึง
128 ต่อไปคงมี ดีแท็ก/เอไอเอส กาแฟ คอยดู
131 ไม่เคยแดก
136 จ่ายค่าโท...ด้วย...หอมกลิ่นกาแฟ..ด้วย.. ไม่ได้ตังค์นางหร้อก
149 เหมือนประชด เคยได้โปรฟรีเป็นกาแฟร้อนตอนหน้าร้อน
152 นี่กาแฟหรือน้ำล้าง..(มือ)
154 กาแฟไม่อร่อยเลย ราคาถูกยังไงก็ไม่คุ้ม
159 อีนี่ก็แอบแพงไป..แดกค่าเนตไม่พอ ยังมากดราคากาแฟอีก 😡
169 ไม่เคยแดก
202 ถ้ามีส่วนลด กาแฟก็โอเคอยู่นะ
215 เลิกใช้กาแฟเพื่อน มาใช้กาแฟคุณภาพได้แล้วครับ
229 เรื่องกาแฟ
247 ขอกา

In [26]:
w2 = Widget(result, 'comment')

ToggleButtons(options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13), value=0)

6 ไม่ ชอบ การ แบ่ง ชนชั้น ในร้านกาแฟ ผู้ถือบัตรอีกแบบ นั่งได้แค่ตรงนี้ ผู้ถือบัตรอีกแบบนั่งตรงนี้ จ่ายราคากาแฟก็เท่ากันไหม
57 เป็นร้านที่แดกกาแฟร้อนทีไร แม่งต้องล้วกลิ้นไหม้เป็นแผลตลอด น้ำร้อนเอ็งจะร้อนไปไหนวะ สงสัยแม่งเอามาจากบ่อในนรกชัวร์ๆ ถ้าเอ็งร้อนแบบนี้
58 ทานที่มหาวิทยาลัยเป็นมุมกาแฟรสชาติอร่อยมาก กาแฟดีมาก ราคาดีรับได้ แต่ถ้าเป็นร้านในห้างราคา over เกินไป
90 กาแฟพอใช้ได้นะ แต่ราคาก็ยังสูงกว่าร้านอื่นๆอยุ่ ลงราคามาหน่อย หรือจัดโทรโมชั่นบ้าง
99 ลองเปลี่ยนไปขายอย่างอื่นที่ไม่ใช่กาแฟดีกว่า
101 กาแฟแม่งแรงสัสมึงจะดีดไปไหน
114 กาแฟจืดสัดๆ เปรี้ยวด้วย ใช้โรบัสต้า ใช่มั้ยมึง
128 ต่อไปคงมี ดีแท็ก/เอไอเอส กาแฟ คอยดู
131 ไม่เคยแดก
136 จ่ายค่าโท...ด้วย...หอมกลิ่นกาแฟ..ด้วย.. ไม่ได้ตังค์นางหร้อก
149 เหมือนประชด เคยได้โปรฟรีเป็นกาแฟร้อนตอนหน้าร้อน
152 นี่กาแฟหรือน้ำล้าง..(มือ)
154 กาแฟไม่อร่อยเลย ราคาถูกยังไงก็ไม่คุ้ม
159 อีนี่ก็แอบแพงไป..แดกค่าเนตไม่พอ ยังมากดราคากาแฟอีก 😡
169 ไม่เคยแดก
202 ถ้ามีส่วนลด กาแฟก็โอเคอยู่นะ
215 เลิกใช้กาแฟเพื่อน มาใช้กาแฟคุณภาพได้แล้วครับ
229 เรื่องกาแฟ
247 ขอกา

In [27]:
w3 = Widget(result, 'tokenized_comment')

ToggleButtons(options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13), value=0)

6 ['ชอบ', 'ร้าน', 'กาแฟ', 'บัตร', 'นั่ง', 'บัตร', 'นั่ง', 'จ่าย', 'ราคา', 'กาแฟ', 'ไหม']
57 ['ร้าน', 'แดก', 'กาแฟ', 'ร้อน', 'แม่ง', 'น้ำ', 'ร้อน', 'ร้อน', 'แม่ง', 'ร้อน']
58 ['ทาน', 'กาแฟ', 'รสชาติ', 'อร่อย', 'กาแฟ', 'ดี', 'ราคา', 'ดี', 'ร้าน', 'ห้าง', 'ราคา']
90 ['กาแฟ', 'ราคา', 'ร้าน']
99 ['ลอง', 'ขาย', 'กาแฟ', 'ดี']
101 ['กาแฟ', 'แม่ง', 'แรง']
114 ['กาแฟ', 'จืด', 'เปรี้ยว']
128 ['ดี', 'กาแฟ', 'ดู']
131 ['แดก']
136 ['จ่าย', 'ค่า', 'หอม', 'กลิ่น', 'กาแฟ']
149 ['เหมือน', 'กาแฟ', 'ร้อน', 'ตอน', 'หน้า', 'ร้อน']
152 ['กาแฟ', 'น้ำ', 'ล้าง', 'มือ']
154 ['กาแฟ', 'อร่อย', 'ราคา', 'คุ้ม']
159 ['แดก', 'ค่า', 'เนต', 'กด', 'ราคา', 'กาแฟ']
169 ['แดก']
202 ['ลด', 'กาแฟ', 'โอเค']
215 ['เลิก', 'กาแฟ', 'กาแฟ', 'คุณภาพ']
229 ['เรื่อง', 'กาแฟ']
247 ['กาแฟ', 'สด']
261 ['กาแฟ', 'น้ำ', 'เปล่า']
271 ['กาแฟ']
273 ['กาแฟ', 'รสชาติ', 'เหมือน']
278 ['จืด', 'อ่า', 'กาก', 'กาแฟ']
285 ['กาแฟ', 'เรื่อง']
288 ['กาแฟ', 'น้ำ', 'ล้าง']
290 ['แพง', 'กาแฟ', 'จืด']
295 ['กาแฟ', 'จืด']
297 ['กาแฟ', 'แรง']
326 ['กาแฟ', 'เหม

In [20]:
seed = 0
compare = 0

a = numpy.array(onehot_corpus.iloc[seed]).reshape(1, -1)
b = numpy.array(onehot_corpus.iloc[compare]).reshape(1, -1)
print(cosine_similarity(a,b))

print(idx_corpus[seed])
print(corpus[seed])
print(idx_corpus[compare])
print(corpus[compare])

[[1.]]
['ร้าน', 'สวย', 'สวย', 'เงิน', 'พนง', 'เรียน', 'ชง', 'กาแฟ']
ร้านสวยมาก สวยจนอยากบอกว่า เอาเงินแต่งร้าน ส่ง พนง ไปเรียนชงกาแฟก่อนเถอะค่ะ
['ร้าน', 'สวย', 'สวย', 'เงิน', 'พนง', 'เรียน', 'ชง', 'กาแฟ']
ร้านสวยมาก สวยจนอยากบอกว่า เอาเงินแต่งร้าน ส่ง พนง ไปเรียนชงกาแฟก่อนเถอะค่ะ
